In [1]:
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [4]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("Process_prev_Data")\
    .getOrCreate()

## load the data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!/content/drive/MyDrive/Data/previous_application.csv.zip

/bin/bash: line 1: /content/drive/MyDrive/Data/previous_application.csv.zip: Permission denied


In [7]:
!unzip -q "/content/drive/MyDrive/Data/previous_application.csv.zip" -d "/content/previous_application.csv"

In [8]:
df_prev = spark.read.csv("/content/previous_application.csv/previous_application.csv", header=True, inferSchema=True)

# Clean Data -> Col null threshold 40% & drop null rows

In [9]:
from pyspark.sql.functions import col, when, count

In [10]:
null_counts = df_prev.select([count(when(col(c).isNull(), c)).alias(c) for c in df_prev.columns])

In [11]:
from pyspark.sql.functions import round

In [12]:
null_percentages = null_counts.select([
    (col(c) / df_prev.count() * 100).alias(c) for c in df_prev.columns
])
null_percentages = null_percentages.select([round(col(c), 2).alias(c) for c in null_percentages.columns])


In [13]:
# Select columns with null percentages below 40% and drop them
cols_to_drop = [c for c, v in null_percentages.first().asDict().items() if v >= 40]
cols_to_drop

['AMT_DOWN_PAYMENT',
 'RATE_DOWN_PAYMENT',
 'RATE_INTEREST_PRIMARY',
 'RATE_INTEREST_PRIVILEGED',
 'NAME_TYPE_SUITE',
 'DAYS_FIRST_DRAWING',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION',
 'NFLAG_INSURED_ON_APPROVAL']

In [14]:
df_prev = df_prev.drop(*cols_to_drop)

In [15]:
print((df_prev.count(), len(df_prev.columns)))

(1670214, 26)


In [16]:
from pyspark.sql.functions import size

In [17]:
# drop the rows with at least one null value
df_prev = df_prev.na.drop()

In [18]:
print((df_prev.count(), len(df_prev.columns)))

(1246320, 26)


# Generate New Features using (prev_features.sh) file

In [19]:
from pyspark.sql.functions import avg

In [20]:
#  AMT_ANNUITY -> avg(AMT_ANNUITY)
prev_filter = df_prev.groupBy("SK_ID_CURR") \
                     .agg(avg("AMT_ANNUITY").alias("PREV_AVG_AMT_ANNUITY"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [21]:
#  AMT_APPLICATION -> avg(AMT_APPLICATION)
prev_filter = df_prev.groupBy("SK_ID_CURR") \
                     .agg(avg("AMT_APPLICATION").alias("PREV_AVG_AMT_APPLICATION"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [22]:
# AMT_CREDIT -> avg(AMT_CREDIT)
prev_filter = df_prev.groupBy("SK_ID_CURR") \
                    .agg(avg("AMT_CREDIT").alias("PREV_AVG_AMT_CREDIT"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [23]:
# AMT_GOODS_PRICE -> avg(AMT_GOODS_PRICE/AMT_APPLICATION)

df_prev = df_prev.withColumn("GOODS_APP_RATIO", col("AMT_GOODS_PRICE") / col("AMT_APPLICATION"))
prev_filter = df_prev.groupBy("SK_ID_CURR") \
                   .agg(avg("GOODS_APP_RATIO").alias("PREV_AVG_GOODS_APP_RATIO"))
df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [24]:
# DAYS_DECISION -> avg(DAYS_DECISION)
prev_filter = df_prev.groupBy("SK_ID_CURR") \
                           .agg(avg("DAYS_DECISION").alias("PREV_AVG_DAYS_DECISION"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [25]:
# CNT_PAYMENT -> mean(CNT_PAYMENT)
prev_filter = df_prev.groupBy("SK_ID_CURR") \
                         .agg(avg("CNT_PAYMENT").alias("PREV_AVG_CNT_PAYMENT"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [26]:
# SK_ID_CURR -> count(SK_ID_CURR)
prev_filter = df_prev.groupBy('SK_ID_CURR').agg(count("*").alias('PREV_COUNT'))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [27]:
# HOUR_APPR_PROCESS_START -> avg(HOUR_APPR_PROCESS_START)
prev_filter = df_prev.groupBy("SK_ID_CURR") \
                          .agg(avg("HOUR_APPR_PROCESS_START").alias("PREV_AVG_HOUR_APPR_PROCESS_START"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

## round the values

In [28]:
# round the vlause
avg_columns = [
    "PREV_AVG_AMT_ANNUITY",
    "PREV_AVG_AMT_CREDIT",
    "PREV_AVG_DAYS_DECISION",
    "PREV_AVG_GOODS_APP_RATIO",
    "PREV_AVG_CNT_PAYMENT",
    "PREV_AVG_HOUR_APPR_PROCESS_START",
    "PREV_AVG_DAYS_DECISION",
]
for col_name in avg_columns:
    df_prev = df_prev.withColumn(col_name, round(col(col_name), 2))

In [29]:
# NAME_CONTRACT_STATUS -> 2 features (approved , refused)

In [30]:
prev_filter = df_prev.groupBy("SK_ID_CURR") \
    .agg(count(when(col("NAME_CONTRACT_STATUS") == "Refused", 1)).alias("REFUSED_STATUS_COUNT"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"REFUSED_STATUS_COUNT": 0})


In [31]:
prev_filter = df_prev.groupBy("SK_ID_CURR") \
    .agg(count(when(col("NAME_CONTRACT_STATUS") == "Approved", 1)).alias("APPROVED_STATUS_COUNT"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"APPROVED_STATUS_COUNT": 0})


In [32]:
# NAME_CONTRACT_TYPE  -> 3 features (all features)

In [33]:
prev_filter = df_prev.groupBy("SK_ID_CURR") \
    .agg(count(when(col("NAME_CONTRACT_TYPE") == "Consumer loans", 1)).alias("Consumer loans"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"Consumer loans": 0})

In [34]:
prev_filter = df_prev.groupBy("SK_ID_CURR") \
    .agg(count(when(col("NAME_CONTRACT_TYPE") == "Revolving loans", 1)).alias("Revolving loans"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"Revolving loans": 0})

In [35]:
prev_filter = df_prev.groupBy("SK_ID_CURR") \
    .agg(count(when(col("NAME_CONTRACT_TYPE") == "Cash loans", 1)).alias("Cash loans"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"Cash loans": 0})

In [36]:
# CODE_REJECT_REASON -> 3 features (XAP, HC , Limit)

In [ ]:
xap_df = df_prev.filter(col("CODE_REJECT_REASON") == "XAP") \
                .groupBy("SK_ID_CURR") \
                .agg(count("*").alias("XAP"))

df_prev = df_prev.join(xap_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"XAP": 0})

In [ ]:
hc_df = df_prev.filter(col("CODE_REJECT_REASON") == "HC") \
               .groupBy("SK_ID_CURR") \
               .agg(count("*").alias("HC"))

df_prev = df_prev.join(hc_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"HC": 0})

In [ ]:
limit_df = df_prev.filter(col("CODE_REJECT_REASON") == "LIMIT") \
                  .groupBy("SK_ID_CURR") \
                  .agg(count("*").alias("LIMIT"))

df_prev = df_prev.join(limit_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"LIMIT": 0})

## drop used features and some unnesesary features (to reduce the overhead)

In [ ]:
columns_to_drop = [
    "NAME_CONTRACT_TYPE",
    "AMT_ANNUITY",
    "AMT_APPLICATION",
    "AMT_CREDIT",
    "AMT_GOODS_PRICE",
    "WEEKDAY_APPR_PROCESS_START",
    "HOUR_APPR_PROCESS_START",
    "NAME_CONTRACT_STATUS",
    "DAYS_DECISION",
    "NAME_PAYMENT_TYPE",
    "CODE_REJECT_REASON",
    "NAME_GOODS_CATEGORY",
    "NAME_PRODUCT_TYPE",
    "SELLERPLACE_AREA",
    "NAME_SELLER_INDUSTRY",
    "CNT_PAYMENT",
    "GOODS_APP_RATIO",
]

# Drop the specified columns
df_prev = df_prev.drop(*columns_to_drop)

In [ ]:
# NAME_PAYMENT_TYPE -> 1 features (cash payment )

In [ ]:
# NOTE : deleted

In [ ]:
# NAME_CLIENT_TYPE -> 2 features (repeater , refreshed)

In [ ]:
refreshed_df = df_prev.filter(col("NAME_CLIENT_TYPE") == "Refreshed") \
                      .groupBy("SK_ID_CURR") \
                      .agg(count("*").alias("Refreshed"))

df_prev = df_prev.join(refreshed_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"Refreshed": 0})

In [ ]:
repeater_df = df_prev.filter(col("NAME_CLIENT_TYPE") == "Repeater") \
                     .groupBy("SK_ID_CURR") \
                     .agg(count("*").alias("Repeater"))

df_prev = df_prev.join(repeater_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"Repeater": 0})

In [ ]:
# NAME_PORTFOLIO -> 3 features (POS , Cash , Cards)

In [ ]:
pos_df = df_prev.filter(col("NAME_PORTFOLIO") == "POS") \
                .groupBy("SK_ID_CURR") \
                .agg(count("*").alias("POS"))

df_prev = df_prev.join(pos_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"POS": 0})

In [ ]:
cash_df = df_prev.filter(col("NAME_PORTFOLIO") == "Cash") \
                 .groupBy("SK_ID_CURR") \
                 .agg(count("*").alias("Cash"))

df_prev = df_prev.join(cash_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"Cash": 0})

In [ ]:
cards_df = df_prev.filter(col("NAME_PORTFOLIO") == "Cards") \
                  .groupBy("SK_ID_CURR") \
                  .agg(count("*").alias("Cards"))

df_prev = df_prev.join(cards_df, on="SK_ID_CURR", how="left")
df_prev = df_prev.fillna({"Cards": 0})

In [ ]:
# CHANNEL_TYPE -> top 3 (Credit and cash offices, Country-wide, Stone)

In [ ]:
# For "Credit and cash offices"
prev_filter = df_prev.filter(df_prev["CHANNEL_TYPE"] == "Credit and cash offices") \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("Credit and cash offices"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

# For "Country-wide"
prev_filter = df_prev.filter(df_prev["CHANNEL_TYPE"] == "Country-wide") \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("Country-wide"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

# For "Stone"
prev_filter = df_prev.filter(df_prev["CHANNEL_TYPE"] == "Stone") \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("Stone"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

df_prev = df_prev.fillna({"Credit and cash offices": 0, "Country-wide": 0, "Stone": 0})

In [ ]:
# PRODUCT_COMBINATION -> top 3 (Cash, POS household with interest, POS mobile with interest)

In [ ]:
# For "Cash"
prev_filter = df_prev.filter(df_prev["PRODUCT_COMBINATION"] == "Cash") \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("Cash"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

# For "POS household with interest"
prev_filter = df_prev.filter(df_prev["PRODUCT_COMBINATION"] == "POS household with interest") \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("POS household with interest"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

# For "POS mobile with interest"
prev_filter = df_prev.filter(df_prev["PRODUCT_COMBINATION"] == "POS mobile with interest") \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("POS mobile with interest"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

df_prev = df_prev.fillna({
    "Cash": 0,
    "POS household with interest": 0,
    "POS mobile with interest": 0
})

In [ ]:
# NAME_YIELD_GROUP -> avg(encoded (NAME_YIELD_GROUP))

In [ ]:
from pyspark.sql import functions as F

In [ ]:
mapping = {
    "XNA": 0,
    "low_normal": 1,
    "middle": 2,
    "high": 3
}

mapping_expr = F.create_map(
    *[F.lit(x).cast("string"), F.lit(y).cast("int") for x, y in mapping.items()]
)

df_prev = df_prev.withColumn("NAME_YIELD_GROUP", mapping_expr[df_prev["NAME_YIELD_GROUP"]])


prev_filter = df_prev.groupBy("SK_ID_CURR") \
    .agg(F.mean("NAME_YIELD_GROUP").alias("PREV_AVG_NAME_YIELD_GROUP"))
df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")

In [ ]:
# FLAG_LAST_APPL_PER_CONTRACT -> sum(0)


In [ ]:
prev_filter = df_prev.filter(df_prev["FLAG_LAST_APPL_PER_CONTRACT"] == 0) \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("ZERO_FLAG_LAST_APPL_PER_CONTRACT"))
df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")
df_prev = df_prev.withColumn("ZERO_FLAG_LAST_APPL_PER_CONTRACT",
                             F.coalesce(df_prev["ZERO_FLAG_LAST_APPL_PER_CONTRACT"], F.lit(0)))

In [ ]:
# NFLAG_LAST_APPL_IN_DAY-> sum(0)

In [ ]:
prev_filter = df_prev.filter(df_prev["NFLAG_LAST_APPL_IN_DAY"] == 0) \
    .groupBy("SK_ID_CURR") \
    .agg(F.count("*").alias("ZERO_NFLAG_LAST_APPL_IN_DAY"))

df_prev = df_prev.join(prev_filter, on="SK_ID_CURR", how="left")
df_prev = df_prev.withColumn("ZERO_NFLAG_LAST_APPL_IN_DAY",
                             F.coalesce(df_prev["ZERO_NFLAG_LAST_APPL_IN_DAY"], F.lit(0)))

In [ ]:
# NAME_CASH_LOAN_PURPOSE -> XAP, other

In [ ]:
prev_filter = df_prev[df_prev["NAME_CASH_LOAN_PURPOSE"] == "XAP"].groupby("SK_ID_CURR").size().reset_index(name="XAP_NAME_CASH_LOAN_PURPOSE")
df_prev = df_prev.merge(prev_filter, on="SK_ID_CURR", how="left")
df_prev["XAP_NAME_CASH_LOAN_PURPOSE"] = df_prev["XAP_NAME_CASH_LOAN_PURPOSE"].fillna(0)

prev_filter = df_prev[(df_prev["NAME_CASH_LOAN_PURPOSE"] != "XAP") & (df_prev["NAME_CASH_LOAN_PURPOSE"] != "XNA")].groupby("SK_ID_CURR").size().reset_index(name="Other_NAME_CASH_LOAN_PURPOSE")
df_prev = df_prev.merge(prev_filter, on="SK_ID_CURR", how="left")
df_prev["Other_NAME_CASH_LOAN_PURPOSE"] = df_prev["Other_NAME_CASH_LOAN_PURPOSE"].fillna(0)

## Drop used Col

In [ ]:
delete_columns = [
    "FLAG_LAST_APPL_PER_CONTRACT",
    "NFLAG_LAST_APPL_IN_DAY",
    "NAME_CASH_LOAN_PURPOSE",
    "CODE_REJECT_REASON",
    "NAME_CLIENT_TYPE",
    "NAME_PORTFOLIO",
    "CHANNEL_TYPE",
    "NAME_YIELD_GROUP",
    "PRODUCT_COMBINATION",
]
df_prev = df_prev.drop(*delete_columns)

## Drop duplicate rows

In [ ]:
df_prev = df_prev.drop('SK_ID_PREV')
df_prev = df_prev.dropDuplicates()

# Save Data

In [ ]:
import os
import shutil
from pathlib import Path
# Define your paths
current_dir = '/content/drive/MyDrive'
data_relative_path = 'Data'
output_dir = os.path.join(current_dir, data_relative_path, "tmp_output")

def save_csv(df, final_csv_path):
    # write df into a temporary folder
    df.coalesce(1).write.option("header", "true").mode("overwrite").csv(output_dir)

    # find the generated part file
    part_file = next(Path(output_dir).glob("part-*.csv"))

    # move and rename
    shutil.move(str(part_file), final_csv_path)

    # delete the temporary folder
    shutil.rmtree(output_dir)


final_csv_path = os.path.join(current_dir, data_relative_path, "processed_previous_application.csv")
save_csv(df_prev, final_csv_path)